## Analyzing pilot boat (only speed)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import src.data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

In [ ]:
from dask.distributed import Client, progress, TimeoutError
client = Client(n_workers=4, threads_per_worker=1, memory_limit='2GB')
client

In [ ]:
reader = pd.read_csv(src.data.path_lots, chunksize=None, iterator=False)
df_raw = reader
df_raw.set_index('Timestamp [UTC]', inplace=True)

In [ ]:
df_raw.describe()

In [ ]:
mask = df_raw.std() > 0
interesting=df_raw.columns[mask]
df = df_raw[interesting].copy()

mask = df['Speed over ground (kts)']>5
df=df.loc[mask]
#df = df.iloc[0:10000]


In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
interesting = [
              'Consumption ME all (L/h)',
               'Economy (L/nm)',
               'Speed over ground (kts)',
              'Engine speed ME1 (rpm)',
              'Engine speed ME2 (rpm)',
              
              ]
data = df[interesting].copy()
data.dropna(how='any', inplace = True)

## Regression

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression

select_k_best = SelectKBest(score_func=f_regression, k=4)
standard_scaler = StandardScaler()
polynomial_features = PolynomialFeatures(degree=2)
linear_regression = LinearRegression()

steps = [
    ('scaler', standard_scaler),
    ('polynomial_features', polynomial_features),
    ('select_k_best', select_k_best),
    ('linear_regression', linear_regression),
]

pipeline_polynomial_scaled_selection = Pipeline(steps=steps)

In [ ]:
#X = data[['Speed over ground (kts)']].copy()
#X['Engine speed'] = (data['Engine speed ME1 (rpm)'] + data['Engine speed ME2 (rpm)'])/2
#y = data['Consumption ME all (L/h)']                        

label_y = 'Consumption ME all (L/h)'
X = data[['Speed over ground (kts)']].copy()
#X['Engine speed'] = (data['Engine speed ME1 (rpm)'] + data['Engine speed ME2 (rpm)'])/2
y = data[label_y]       

In [ ]:
from sklearn.model_selection import GridSearchCV
import joblib

# Decide how many k
max_degree = 5
no_features = len(X.columns)
polynomial_features = PolynomialFeatures(degree=max_degree)
polynomial_features.fit(np.array([np.arange(no_features)]))
max_k = len(polynomial_features.get_feature_names())


# define the grid
grid = dict()
grid['select_k_best__k'] = [i for i in range(1, max_k+1)]
grid['polynomial_features__degree'] = [i for i in range(1, max_degree+1)]

from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)

# define the grid search
search = GridSearchCV(estimator=pipeline_polynomial_scaled_selection, param_grid=grid, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
#search = GridSearchCV(estimator=pipeline_polynomial_scaled_selection, param_grid=grid, scoring='r2', n_jobs=-1, cv=cv)
# perform the search
with joblib.parallel_backend('dask'):
    search_result = search.fit(X, y)

In [ ]:
model = search_result.best_estimator_

In [ ]:
search_result.cv_results_['mean_test_score']

In [ ]:
search_result.cv_results_['std_test_score']

In [ ]:
cv_results_ = search_result.cv_results_
df_search = pd.DataFrame()
for param, mean_test_score, std_test_score in zip(cv_results_['params'],cv_results_['mean_test_score'],cv_results_['std_test_score']):
    r_ = pd.Series(param)
    r_['mean_test_score'] = mean_test_score
    r_['std_test_score'] = std_test_score
    df_search = df_search.append(r_, ignore_index=True)


In [ ]:
fig,ax=plt.subplots()

for k, group in df_search.groupby('polynomial_features__degree'):
    
    label = 'k:%i' % k
    ax.errorbar(group['select_k_best__k'], group['mean_test_score'], yerr=group['std_test_score'], linestyle='-', marker='.', label=label)
    
ax.grid()
ax.legend()    

In [ ]:
fig,ax=plt.subplots()

mask = ((df_search['select_k_best__k'] >= 4) &
        (df_search['polynomial_features__degree'] >= 1)
       )
df_search_zoom = df_search.loc[mask].copy()

for k, group in df_search_zoom.groupby('polynomial_features__degree'):
    
    label = 'k:%i' % k
    ax.errorbar(group['select_k_best__k'], group['mean_test_score'], yerr=group['std_test_score'], linestyle='-', marker='.', label=label)
    
ax.grid()
ax.legend() 

In [ ]:
select_k_best = SelectKBest(score_func=f_regression, k=6)
standard_scaler = StandardScaler()
polynomial_features = PolynomialFeatures(degree=5)
linear_regression = LinearRegression()

steps = [
    ('scaler', standard_scaler),
    ('polynomial_features', polynomial_features),
    ('select_k_best', select_k_best),
    ('linear_regression', linear_regression),
]

model = Pipeline(steps=steps)
model.fit(X=X, y=y)

In [ ]:
%%timeit
cv2 = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(estimator=model, X=X, y=y, scoring='r2', cv=cv2, n_jobs=-1)

fig,ax = plt.subplots()
ax.hist(scores);
ax.set_xlabel('score')
ax.set_ylabel('occurances')
ax.set_title('Histogram over cross validations');

In [ ]:
%%timeit

cv2 = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
with joblib.parallel_backend('dask'):
    scores = cross_val_score(estimator=model, X=X, y=y, scoring='r2', cv=cv2, n_jobs=-1)

fig,ax = plt.subplots()
ax.hist(scores);
ax.set_xlabel('score')
ax.set_ylabel('occurances')
ax.set_title('Histogram over cross validations');

In [ ]:
result = X.copy()
result['y_true'] = y
result['y_pred'] = model.predict(X=X)

fig,axes=plt.subplots(ncols=len(X.columns))
for x,ax in zip(X, [axes]):
    result.plot(x=x, y='y_true', ax=ax, style='o', alpha=0.1)
    result.plot(x=x, y='y_pred', ax=ax, style='.')
    
axes.set_ylabel(label_y)

In [ ]:
from src.visualization.polynom import model_to_string,model_to_sympy
features = X.columns
print(model_to_string(model=model, feature_names=features))

In [ ]:
features_latex = {
    'Speed over ground (kts)' : r'V_og',
    }
latex_features = [features_latex[key] for key in features]

eq = model_to_sympy(model=model, feature_names=latex_features, label='V')
eq